In [10]:
import argparse
import os
import time

from src.naver_movie_scraper.naver_movie_scraper import scrap_basic
from src.naver_movie_scraper.naver_movie_scraper import scrap_casting
from src.naver_movie_scraper.naver_movie_scraper import scrap_bestscripts
from src.naver_movie_scraper.naver_movie_scraper import scrap_comments
from src.naver_movie_scraper.naver_movie_scraper import save_list_of_dict
from src.naver_movie_scraper.naver_movie_scraper import save_json
import src.mongos as mg

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Const

In [26]:
def add_idx_to_list(idx, data):
    for d in data:
        d['idx'] = idx
    return data


def scrap(idx, db, casting=True, bestscripts=True, comments=True, limit=3, sleep=0.1):
    # basic
#     save_json(scrap_basic(idx), '{}/meta/{}.json'.format(directory, idx))
    mg.to_mongo(scrap_basic(idx), db, 'meta', dtype='dict')
    print('scraped {} basic'.format(idx))

    # castings
    if casting:
        castings = scrap_casting(idx)
        for key in ['actors', 'directors', 'staffs']:
            if castings.get(key, []):
#                 save_list_of_dict(castings[key], '{}/{}/{}'.format(directory, key, idx))
                data = castings[key]
                add_idx_to_list(idx, data)
                mg.to_mongo(data, db, key, dtype='list')
        print('scraped {} casting'.format(idx))

    # best scripts
    if bestscripts:
        scripts = scrap_bestscripts(idx, limit, sleep)
        if scripts:
#             save_list_of_dict(scripts, '{}/bestscripts/{}'.format(directory, idx))
            add_idx_to_list(idx, scripts)
            mg.to_mongo(scripts, db, 'scripts', dtype='list')
        print('scraped {} best scripts'.format(idx))

    # comments
    if comments:
        comments_ = scrap_comments(idx, limit, sleep)
        if comments_:
#             save_list_of_dict(comments_, '{}/comments/{}'.format(directory, idx))
            add_idx_to_list(idx, comments_)
            mg.to_mongo(comments_, db, 'comments', dtype='list')
        print('scraped {} comments'.format(idx))

    print('')
    time.sleep(sleep)

In [27]:
idxs = ['44885', '62560', '49008', '69989', '76348', '72363', '70254', '95873', '96327', '97857', '98438', '92064', '122527',
        '125459', '127398', '135874', '134898', '137326', '136315', '144330', '132623', '136900',]

db = 'avengers_endgame'
limit = -1
sleep = 0.3
casting = True
bestscripts = True
comments = True

exceptions = []
for idx in idxs:
    try:
        scrap(idx, db, casting, bestscripts, comments, limit, sleep)
    except Exception as e:
        print('movie id = {}'.format(idx))
        print(e)
        exceptions.append({'idx': idx, 'error': str(e)})
    if exceptions:
        print('Exist {} exceptions'.format(len(exceptions)))


if exceptions:
    mg.to_mongo(exceptions, db, 'log', dtype='list')

# with open('./log', 'w', encoding='utf-8') as f:
#     if not exceptions:
#         f.write('Information of all movies were scraped successfully.\n')
#     else:
#         f.write('Exist exceptions\n\n')
#         for idx, e in exceptions:
#             f.write('movie id = {}'.format(idx))
#             f.write('{}\n'.format(e))

Inserted row: 5cc7f7d68f455e5b81e946a0
scraped 44885 basic
Inserted rows: 11
Inserted rows: 1
Inserted rows: 68
scraped 44885 casting
Inserted rows: 1518
scraped 44885 best scripts
  movie 44885, after, 2098 / 2098 done
  movie 44885, before, 57 / 57 done
Inserted rows: 20228
scraped 44885 comments

Inserted row: 5cc7f9bd8f455e5b81e99be9
scraped 62560 basic
Inserted rows: 11
Inserted rows: 1
Inserted rows: 70
scraped 62560 casting
Inserted rows: 240
scraped 62560 best scripts
  movie 62560, after, 433 / 433 done
  movie 62560, before, 32 / 32 done
Inserted rows: 3063
scraped 62560 comments

Inserted row: 5cc7fa178f455e5b81e9a929
scraped 49008 basic
Inserted rows: 14
Inserted rows: 1
Inserted rows: 75
scraped 49008 casting
Inserted rows: 1013
scraped 49008 best scripts
  movie 49008, after, 1296 / 1296 done
  movie 49008, before, 145 / 145 done
Inserted rows: 11878
scraped 49008 comments

Inserted row: 5cc7fb468f455e5b81e9dbe5
scraped 69989 basic
Inserted rows: 50
Inserted rows: 1
Inser

KeyboardInterrupt: 

In [ ]:
# main
# def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--directory', type=str, default='./output/', help='Output directory')
#     parser.add_argument('--begin_idx', type=int, default=134963, help='Index of first movie')
#     parser.add_argument('--end_idx', type=int, default=134963, help='Index of last movie')
#     parser.add_argument('--specific_idx', type=str, default='', help='Index of specific movies')
#     parser.add_argument('--limit', type=int, default=3, help='Page limitation for comments & best scripts')
#     parser.add_argument('--sleep', type=float, default=0.1, help='Sleep time per each page in comments & best scripts')
#     parser.add_argument('--casting', dest='casting', action='store_true')
#     parser.add_argument('--bestscripts', dest='bestscripts', action='store_true')
#     parser.add_argument('--comments', dest='comments', action='store_true')

#     args = parser.parse_args()
#     directory = args.directory
#     begin_idx = args.begin_idx
#     end_idx = args.end_idx
#     specific_idx = args.specific_idx
#     limit = args.limit
#     sleep = args.sleep
#     casting = args.casting
#     bestscripts = args.bestscripts
#     comments = args.comments

#     for subdir in ['meta', 'actors', 'directors', 'staffs', 'bestscripts', 'comments']:
#         path = '{}/{}/'.format(directory, subdir)
#         if not os.path.exists(path) or os.path.isfile(path):
#             os.makedirs(path)

#     idxs = range(begin_idx, end_idx + 1)
#     if specific_idx:
#         idxs = [int(idx) for idx in specific_idx.split('_')]

#     exceptions = []
#     for idx in idxs:
#         try:
#             scrap(idx, directory, casting, bestscripts, comments, limit, sleep)
#         except Exception as e:
#             print('movie id = {}'.format(idx))
#             print(e)
#             exceptions.append((idx, str(e)))
#         if exceptions:
#             print('Exist {} exceptions'.format(len(exceptions)))

#     with open('./log', 'w', encoding='utf-8') as f:
#         if not exceptions:
#             f.write('Information of all movies were scraped successfully.\n')
#         else:
#             f.write('Exist exceptions\n\n')
#             for idx, e in exceptions:
#                 f.write('movie id = {}'.format(idx))
#                 f.write('{}\n'.format(e))


# if __name__ == '__main__':
#     main()